### **Explication de la Distorsion de l'EATS dans un Jet en Accélération/Décélération**

Lorsqu’un jet de GRB (une "coquille" de matière, ou *shell*) **accélère ou décélère**, la forme de la **Equal Arrival Time Surface (EATS)** se déforme par rapport au cas simplifié d’un jet à vitesse constante.

---

#### **1. Cas de base : EATS pour un jet à vitesse constante (Γ fixe)**
- La surface EATS est un **ellipsoïde allongé** (par effet Doppler).  
- Tous les photons émis sur cette surface arrivent à l’observateur **en même temps**, car les retards dus aux angles sont compensés par la vitesse relativiste.  

---

#### **2. Jet en Accélération (Γ augmente)**
- **Phénomène** : Le jet devient **de plus en plus rapide** avec le temps.  
- **Conséquence sur l’EATS** :  
  - Les photons émis **à des latitudes élevées** (loin de l’axe) doivent partir **plus tôt** pour "rattraper" l’accélération du jet.  
  - La surface EATS est **étirée vers l’extérieur** (déviation par rapport à l’ellipsoïde).  
- **Exemple** : Si le jet est alimenté continuellement par un moteur central (ex : accrétion prolongée).  

---

#### **3. Jet en Décélération (Γ diminue)**
- **Phénomène** : Le jet ralentit (ex : interaction avec le milieu circumburst).  
- **Conséquence sur l’EATS** :  
  - Les photons émis **à des latitudes élevées** doivent partir **plus tard** (car le jet ralentit et "attend" les photons).  
  - La surface EATS est **comprimée vers l’intérieur**.  
- **Exemple** : Phase de choc externe lors de l’afterglow.  

---

#### **4. Implications Physiques**
- **Courbe de lumière** :  
  - Une EATS étirée (accélération) peut **lisser les variations rapides**.  
  - Une EATS comprimée (décélération) peut **accentuer les pics**.  
- **Spectre observé** :  
  - La distorsion change l’angle effectif de vision → modifie le **boost Doppler** et donc l’énergie apparente des photons.  

---

#### **5. Modélisation Mathématique (Intuition)**
Pour un jet en accélération :  
- Le temps de vol des photons doit satisfaire :  
  
  $t_{\text{obs}} = \int \frac{1 - \beta(t) \cos \theta}{\beta(t) c} \, \mathrm{d}r \quad \text{(intégrale le long du trajet)}  $
  
  où \(\beta(t)\) augmente (accélération) ou diminue (décélération).  

---

#### **6. Analogie**
- **Accélération** : Comme une voiture qui accélère → Vous devez lancer une balle **plus tôt** pour qu’elle atteigne le passager à temps.  
- **Décélération** : Comme une voiture qui freine → Vous lancez la balle **plus tard**.  

---

### **Résumé Graphique**
```
Jet à Γ constant → EATS = Ellipsoïde  
Jet accélérant   → EATS étirée (photons partis tôt à θ élevé)  
Jet décélérant   → EATS comprimée (photons partis tard à θ élevé)  
```

**Pourquoi est-ce crucial ?**  
- Sans corriger cette distorsion, on **surestime la taille réelle du jet** ou on **rate des signatures physiques** (ex : émissions géantes latentes).  

Un exemple concret avec des équations détaillées vous intéresse ? 😊

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import fsolve
import astropy.constants as cst
import astropy.units as u

# Constantes physiques
m_p = cst.m_p.to('g').value    # masse du proton [g]
m_e = cst.m_e.to('g').value    # masse de l'électron [g]
c = cst.c.to('cm/s').value     # vitesse de la lumière [cm/s]
sigma_T = cst.sigma_T.to('cm^2').value  # section efficace de Thomson [cm^2]
q_e = 4.80320425e-10           # charge de l'électron [esu]



class OnAxisJet:
    def __init__(self):
        # Paramètres du jet article
        self.E_iso = 1e53       # énergie isotrope [erg]
        self.n_0 = 1e-3        # densité du milieu [cm^-3]
        self.epsilon_e = 0.1    # fraction d'énergie dans les électrons
        self.epsilon_B = 1e-4   # fraction d'énergie dans le champ B
        self.p = 2.2            # indice spectral des électrons
        self.theta_obs = 0.0    # angle d'observation (0 = dans l'axe)


    #Facteur de Lorentz du choc basé sur l'interpolation équation (1)
    def gamma_shock(self, t):

      # Coefficients BM et ST
      C_BM = np.sqrt(17 * self.E_iso / (8 * np.pi * self.n_0 * m_p * c**5))  # [s^(-3/2)]
      C_ST = (2/5) * 1.15 * (self.E_iso / (self.n_0 * m_p * c**5))**(1/5)     # [s^(-3/5)]

      # Valeur limite
      Gamma_max = 100

      # Calcul de Gamma beta
      gamma2_beta2 = min((C_BM**2) * t**(-3),(C_ST**2) * t**(-6/5),Gamma_max**2)

      # Calcul de β_shock
      beta_sh = np.sqrt(gamma2_beta2 / (1 + gamma2_beta2))

      # Calcul de Γ_shock
      Gamma_sh = 1 / np.sqrt(1 - beta_sh**2)

      return Gamma_sh



    #Rayon du choc
    def shock_radius(self, t):
        def beta_sh(t_prime):
            Gamma = self.gamma_shock(t_prime)
            if Gamma <= 1:
                return 0.0
            return np.sqrt(1 - 1 / Gamma**2)

        R = c * quad(beta_sh, 0, t)[0]
        return R

    #Propriétés en aval du choc
    def downstream_properties(self, t):
        Gamma_sh = self.gamma_shock(t)

        # Approximation ultra-relativiste (valable pour Gamma_sh >> 1)
        Gamma = Gamma_sh / np.sqrt(2)              # facteur de Lorentz du fluide
        n_prime = 4 * Gamma_sh * self.n_0          # densité en aval [cm^-3]
        e_prime = 4 * Gamma_sh**2 * self.n_0 * m_p * c**2  # densité d'énergie [erg/cm^3]

        return Gamma, n_prime, e_prime


    #Equation 31 t= T + muR/c
    def solve_emission_time(self, T_obs, mu):
      def equation(t):
          R = jet.shock_radius(t)
          return  mu * R / c + T_obs - t
      return fsolve(equation, T_obs)[0]



T_obs = 1e5
mu = 0.8

jet = OnAxisJet()
t_emit = jet.solve_emission_time(T_obs, mu)
R_emit = jet.shock_radius(t_emit)
T_check = t_emit - mu * R_emit / c



print(f"Résolution de l'équation 31")
print(f"mu = {mu}")
print(f"T_obs = {T_obs:.2e} s")
print(f"t_emit = {t_emit:.8e} s")
print(f"R(t_emit) = {R_emit:.2e} cm")
print(f"Vérification : t_emit - mu * R / c = {T_check:.2e} s")
%timeit jet.solve_emission_time(T_obs, mu)


Résolution de l'équation 31
mu = 0.8
T_obs = 1.00e+05 s
t_emit = 4.99817785e+05 s
R(t_emit) = 1.50e+16 cm
Vérification : t_emit - mu * R / c = 1.00e+05 s
6.16 ms ± 813 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
class SimpleBetaJet:
    def __init__(self, beta0=1, alpha=0.2):
        self.beta0 = beta0
        self.alpha = alpha

    def beta(self, t):
        return self.beta0 * t**self.alpha

    def shock_radius(self, t):
        # Intégration de beta(t) pour obtenir R(t)
        def beta_func(t_prime):
            b = self.beta(t_prime)
            return np.clip(b, 0, 1)  # pour éviter β > 1
        R = c * quad(beta_func, 0, t)[0]
        return R

    def solve_emission_time(self, T_obs, mu):
        def equation(t):
            R = self.shock_radius(t)
            return mu * R / c + T_obs - t
        return fsolve(equation, T_obs)[0]

In [43]:
T_obs = 1e5
mu = 0.8

jet_poly = SimpleBetaJet(beta0=1, alpha=0.2)
t_emit_poly = jet_poly.solve_emission_time(T_obs, mu)
R_emit_poly = jet_poly.shock_radius(t_emit_poly)
T_check_poly = t_emit_poly - mu * R_emit_poly / c

print(f"Test avec β(t) polynomial")
print(f"mu = {mu}")
print(f"T_obs = {T_obs:.2e} s")
print(f"t_emit = {t_emit_poly:.8e} s")
print(f"R(t_emit) = {R_emit_poly:.2e} cm")
print(f"Vérification : t_emit - mu * R / c = {T_check_poly:.5e} s")

Test avec β(t) polynomial
mu = 0.8
T_obs = 1.00e+05 s
t_emit = 5.00000000e+05 s
R(t_emit) = 1.50e+16 cm
Vérification : t_emit - mu * R / c = 1.00000e+05 s
